# "Quick look" plots - LZIFU
---
Use `spaxelsleuth` to make nice summary plots for a given galaxy. 

In [3]:
%matplotlib widget

In [4]:
# Imports
import sys
import os 
import numpy as np
import pandas as pd
from astropy.visualization import hist
from astropy.io import fits

from spaxelsleuth.loaddata.sami import load_sami_df
from spaxelsleuth.loaddata.lzifu import load_lzifu_df
from spaxelsleuth.plotting.plot2dmap import plot2dmap
from spaxelsleuth.plotting.sdssimg import plot_sdss_image
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram, plot_BPT_lines
from spaxelsleuth.plotting.plottools import vmin_fn, vmax_fn, label_fn, cmap_fn, component_colours
from spaxelsleuth.plotting.plotgalaxies import plot2dscatter, plot2dhistcontours

import matplotlib
from matplotlib import rc, rcParams
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from IPython.core.debugger import Tracer

rc("text", usetex=False)
rc("font",**{"family": "serif", "size": 12})
rcParams["savefig.bbox"] = "tight"
rcParams["savefig.format"] = "pdf"
plt.ion()
plt.close("all")


In [5]:
###########################################################################
# Paths
###########################################################################
assert "SAMI_DIR" in os.environ, "Environment variable SAMI_DIR is not defined!"
sami_data_path = os.environ["SAMI_DIR"]
assert "SAMI_DATACUBE_DIR" in os.environ, "Environment variable SAMI_DATACUBE_DIR is not defined!"
sami_datacube_path = os.environ["SAMI_DATACUBE_DIR"]

In [6]:
###########################################################################
# Options
###########################################################################
savefigs = True
bin_type = "default"    # Options: "default" or "adaptive" for Voronoi binning
ncomponents = "recom"   # Options: "1" or "recom"
eline_SNR_min = 5       # Minimum S/N of emission lines to accept

In [7]:
###########################################################################
# Load the SAMI sample
###########################################################################
df = load_sami_df(ncomponents=ncomponents,
                  bin_type=bin_type,
                  eline_SNR_min=eline_SNR_min,
                  correct_extinction=True)

In load_sami_df(): Loading DataFrame...
In load_sami_df(): Finished!


In [63]:
###########################################################################
# Select a galaxy
###########################################################################
gal = 272990
assert gal in df["ID"].values, f"{gal} not found in DataFrame!"

###########################################################################
# Load the LZIFU DataFrame
# NOTE: NOT using the DF with extinction correction, because it may be a bit dodgy for individual line components. 
# For the purposes of just checking the fluxes in the emission lines, it doens't really matter anyway.
###########################################################################
df_gal = load_lzifu_df(gal=gal, 
                       bin_type=bin_type, 
                       ncomponents=ncomponents,
                       eline_SNR_min=eline_SNR_min,
                       correct_extinction=False)    


In load_lzifu_df(): Loading LZIFU DataFrame for galaxy 272990...
In load_lzifu_df(): Finished!


In [66]:
###########################################################################
# Create a collage figure displaying various quantities of interest for 
# this galaxy
###########################################################################
# layout:
# SDSS image | number of components | BT classifications
# delta sigma: 1 | 2 | 3
# EW: 1 | 2 | 3
# BPT
# Emission lines 
# WHAN | WHAV

fig, axs = plt.subplots(nrows=9, ncols=3, figsize=(13, 35))
fig.subplots_adjust(wspace=0.05, hspace=0.05)

#///////////////////////////////////////////////////////////////////////////////////////////////////////
# SDSS image 
ax = plot_sdss_image(df_gal, ax=axs[0][0])
if ax is not None:
    ax.set_title(f"GAMA{gal}")
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)

#///////////////////////////////////////////////////////////////////////////////////////////////////////
# Number of components
_, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
          PA_deg=0,
          col_z="Number of components",
          ax=axs[0][1], 
          plot_colorbar=True, cax=None, cax_orientation="horizontal", 
          show_title=False)
lon = ax.coords[0]
lon.set_ticklabel_visible(False)
lat = ax.coords[1]
lat.set_ticklabel_visible(False)

#///////////////////////////////////////////////////////////////////////////////////////////////////////
# BPT classifications 
_, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
          PA_deg=0,
          col_z="BPT (numeric) (total)",
          ax=axs[0][2], 
          plot_colorbar=True, cax=None, cax_orientation="vertical", 
          show_title=False)
lon = ax.coords[0]
lon.set_ticklabel_visible(False)
lat = ax.coords[1]
lat.set_ticklabel_visible(False)

#///////////////////////////////////////////////////////////////////////////////////////////////////////
# v_gas
for nn in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"v_gas (component {nn + 1})",
              ax=axs[1][nn], 
              plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
              vmin=-200, vmax=+200,
              show_title=False)
    ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[1][nn].transAxes, verticalalignment="top")
    if nn > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)

#///////////////////////////////////////////////////////////////////////////////////////////////////////
# sigma_gas
for nn in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"sigma_gas (component {nn + 1})",
              ax=axs[2][nn], 
              plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
              show_title=False)
    ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[2][nn].transAxes, verticalalignment="top")
    if nn > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)
    
#///////////////////////////////////////////////////////////////////////////////////////////////////////
# HALPHA
for nn in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"HALPHA (component {nn + 1})",
              ax=axs[3][nn], 
              plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
              vmin=0, vmax=2.0,
              show_title=False)
    ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[3][nn].transAxes, verticalalignment="top")
    if nn > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)
#///////////////////////////////////////////////////////////////////////////////////////////////////////
# [NII]
for nn in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"NII6583 (component {nn + 1})",
              ax=axs[4][nn], 
              plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
              vmin=0, vmax=0.75, cmap="viridis",
              show_title=False)
    ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[4][nn].transAxes, verticalalignment="top")
    if nn > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)

#///////////////////////////////////////////////////////////////////////////////////////////////////////
# [SII]
for nn in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"SII6716 (component {nn + 1})",
              ax=axs[5][nn], 
              plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
              vmin=0, vmax=0.75, cmap="viridis",
              show_title=False)
    ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[5][nn].transAxes, verticalalignment="top")
    if nn > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)
    
#///////////////////////////////////////////////////////////////////////////////////////////////////////
# HBETA
for nn in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"HBETA (component {nn + 1})",
              ax=axs[6][nn], 
              plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
              vmin=0, vmax=0.75, cmap="viridis",
              show_title=False)
    ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[6][nn].transAxes, verticalalignment="top")
    if nn > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)
    
#///////////////////////////////////////////////////////////////////////////////////////////////////////
# [OIII]
for nn in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"OIII5007 (component {nn + 1})",
              ax=axs[7][nn], 
              plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
              vmin=0, vmax=0.75, cmap="viridis",
              show_title=False)
    ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[7][nn].transAxes, verticalalignment="top")
    if nn > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)
    
#///////////////////////////////////////////////////////////////////////////////////////////////////////
# EW 
for nn in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"HALPHA EW (component {nn + 1})",
              ax=axs[8][nn], 
              plot_colorbar=True if nn == 2 else False, cax=None, cax_orientation="vertical", 
              show_title=False)
    ax.text(s=f"Component {nn + 1}", x=0.1, y=0.95, transform=axs[8][nn].transAxes, verticalalignment="top")
    if nn > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
###########################################################################
# "Lag" plot
###########################################################################
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
fig = plot2dhistcontours(df,
                         col_x="v_gas (component 1)",
                         col_y="v_gas (component 2)",
                         col_z="count", log_z=False, cmap="gray_r", alpha=0.4,
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300, 
                         ax=axs[0],
                         plot_colorbar=False,
                         nbins=100)
fig = plot2dhistcontours(df,
                         col_x="v_*",
                         col_y="v_gas (component 2)",
                         col_z="count", log_z=False, cmap="gray_r", alpha=0.4,
                         xmin=-300, xmax=300, 
                         ymin=-300, ymax=300,
                         ax=axs[1],
                         plot_colorbar=False,
                         nbins=100)
axs = fig.get_axes()
 
_ = plot2dscatter(df_gal, col_x="v_gas (component 1)", col_y="v_gas (component 2)", col_z="sigma_gas (component 2)",
                  xmin=-300, xmax=300, ymin=-300, ymax=300, 
                  vmax=150,
                  plot_colorbar=False,
                  ax=axs[0])

_ = plot2dscatter(df_gal, col_x="v_*", col_y="v_gas (component 2)", col_z="sigma_gas (component 2)",
                  xmin=-300, xmax=300, ymin=-300, ymax=300, 
                  vmax=150,
                  plot_colorbar=True,
                  ax=axs[1])
cax = fig.get_axes()[-1]

# Decorations 
[ax.grid() for ax in axs]
[ax.set_xlabel(ax.get_xlabel() + " (component 1)") for ax in axs]
axs[0].set_ylabel(axs[0].get_ylabel() + " (component 2)")
axs[1].set_ylabel("")
cax.set_ylabel(cax.get_ylabel() + " (component 2)")
# axs[1].set_yticklabels([])
[ax.plot([-300, 300], [-300, 300], color="k", lw=2, ls="--") for ax in axs]
[ax.plot([-300, 300], [0, 0], color="k") for ax in axs]
[ax.plot([0, 0], [-300, 300], color="k") for ax in axs]
axs[0].axvline(df_gal.loc[df_gal["x, y (pixels)"] == (25, 25), "v_gas (component 1)"].values[0], ls="--", label=r"$v_{\rm gas\, (25,25)}$")
axs[0].axhline(df_gal.loc[df_gal["x, y (pixels)"] == (25, 25), "v_gas (component 2)"].values[0], ls="--")
axs[0].legend()
axs[1].axhline(df_gal.loc[df_gal["x, y (pixels)"] == (25, 25), "v_gas (component 2)"].values[0], ls="--", label=r"$v_{\rm gas\, (25,25)}$")
axs[1].axvline(df_gal.loc[df_gal["x, y (pixels)"] == (25, 25), "v_*"].values[0], ls=":", label=r"$v_{\rm *\, (25,25)}$")
axs[1].legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
###########################################################################
# Optical diagnostic (i.e. BPT) diagram
###########################################################################
fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True, figsize=(12, 4))
markers = ["o", ">", "D"]
col_z = "r/R_e"
for cc, col_x in enumerate(["log N2", "log S2", "log O1"]):
    # Plot full SAMI sample
    plot2dhistcontours(df=df, 
                       col_x=f"{col_x} (total)",
                       col_y=f"log O3 (total)", col_z="count", log_z=True,
                       alpha=0.5, cmap="gray_r",
                       ax=axs[cc], plot_colorbar=False)

    # Plot measurements for this galaxy
    for nn in range(3):
        plot2dscatter(df=df_gal,
                      col_x=f"{col_x} (component {nn + 1})",
                      col_y=f"log O3 (component {nn + 1})",
                      col_z=col_z,
                      cax=cax,
                      marker=markers[nn], ax=axs[cc], 
                      markersize=20, 
                      markeredgecolor="black",
                      plot_colorbar=True if cc == 2 else False)

    # axis limits
    axs[cc].set_xlim([np.nanmin([vmin_fn(col_x), df_gal[f"{col_x} (total)"].min() - 0.1]),
                      np.nanmax([vmax_fn(col_x), df_gal[f"{col_x} (total)"].max() + 0.1])])
    # axis limits
    axs[cc].set_ylim([np.nanmin([vmin_fn("log O3"), df_gal[f"log O3 (total)"].min() - 0.1]),
                      np.nanmax([vmax_fn("log O3"), df_gal[f"log O3 (total)"].max() + 0.1])])

# Decorations
[ax.grid() for ax in axs]
[ax.set_ylabel("") for ax in axs[1:]]
[ax.set_yticklabels([]) for ax in axs[1:]]
[ax.set_xticks(ax.get_xticks()[:-1]) for ax in axs[:-1]]
[ax.collections[0].set_rasterized(True) for ax in axs]


/home/u5708159/python/Modules/spaxelsleuth/plotting/plottools.py:707: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AssertionError: log N2 (component 1) is not a valid column!

In [42]:
plt.close("all")